In [98]:
import pandas as pd
import json
import re

In [21]:
#찬희 json로드
file_path = r'D:\Study\Python\Jupyter\law_json_data\clean_laws_chanhee.json'
with open(file_path, 'r', encoding="UTF-8") as jsonfile:
    json_data_chan = json.load(jsonfile)    


In [22]:
#유정 json로드
file_path = r'D:\Study\Python\Jupyter\law_json_data\cleaned_laws_yujeong.json'
with open(file_path, 'r', encoding="UTF-8") as jsonfile:
    json_data_yu = json.load(jsonfile)  

In [23]:
#새하 json로드
file_path = r'D:\Study\Python\Jupyter\law_json_data\clean_laws_saeha.json'
with open(file_path, 'r', encoding="UTF-8") as jsonfile:
    json_data_sae = json.load(jsonfile)  

In [24]:
#한슬 json로드
file_path = r'D:\Study\Python\Jupyter\law_json_data\clean_laws_hanseul.json'
with open(file_path, 'r', encoding="UTF-8") as jsonfile:
    json_data_han = json.load(jsonfile)  

In [29]:
json_data_chan_list = json_data_chan["laws"]
json_data_yu_list = json_data_yu["laws"]
json_data_sae_list = json_data_sae["laws"]
json_data_han_list = json_data_han["laws"]

In [31]:
#하나로 합치기 
laws_data_list = json_data_chan_list+json_data_yu_list+json_data_sae_list+json_data_han_list
print(len(laws_data_list))

158957


In [42]:
laws_data_list_cut = laws_data_list[0:100]

In [58]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from konlpy.tag import Okt
okt=Okt() #형태소 분해 객체 생성

In [59]:
tagged_data = [TaggedDocument(words=okt.morphs(_d), tags=[str(i)]) for i, _d in enumerate(laws_data_list_cut)]
for i in tagged_data:
      print(i)

TaggedDocument(['목적', '이', '법', '은', '1027', '법난', '과', '관련', '하여', '피해', '를', '입은', '자', '와', '불교', '계', '의', '명예', '를', '회복', '시켜', '줌', '으로써', '인권', '신장', '과', '국민', '화합', '에', '이바지', '함', '을', '목적', '으로', '한다'], ['0'])
TaggedDocument(['정의', '이', '법', '에서', '사용', '하는', '용어', '의', '정의', '는', '다음', '과', '같다', '1027', '법난', '이란', '1980년', '10월', '계엄', '사령부', '의', '합동', '수사', '본부', '합동', '수', '사단', '이', '불교', '계', '정화', '를', '명분', '으로', '대', '한', '불교', '조계종', '의', '승려', '및', '불교', '관련', '자', '를', '강제', '로', '연행', '수사', '하고', '포고령', '위반', '수배', '자', '및', '불순', '분자', '를', '검거', '한다는', '구실', '로', '군경', '합동', '으로', '전국', '의', '사찰', '및', '암자', '등', '을', '수색', '한', '사건', '을', '말', '한다', '피해자', '란', '1027', '법난', '으로', '인하여', '사망', '또는', '상이', '를', '입은', '자를', '말', '한다', '피해', '종교', '단체', '란', '1027', '법난', '으로', '인하여', '종교', '적', '존엄성', '과', '명예', '를', '훼손', '당', '한', '대', '한', '불교', '조계종', '및', '1027', '법난', '당시', '피해자', '가', '소속', '된', '사찰', '을', '말', '한다'], ['1'])
TaggedDocument(['피해자', '및

In [60]:
# 모델 설계
max_epochs = 100
model = Doc2Vec(vector_size=20,
                alpha=0.025,
                min_alpha=0.00025,
                min_count=1,
                dm=1)

In [61]:
# doc2vec이 단어 사전을 만들어냄
model.build_vocab(tagged_data)

In [67]:
for epoch in range(max_epochs):
    model.train(tagged_data,total_examples=model.corpus_count,epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")

In [99]:
load_model= Doc2Vec.load("d2v.model")

test_string = "소방 대원 관련"
tokened_test_string = okt.nouns(test_string) # input을 형태소 분석을 실시
tokened_test_string


['소방', '대원', '관련']

In [100]:
test_vector = load_model.infer_vector(tokened_test_string)
test_vector

array([-0.03374168, -0.0308762 ,  0.00688379, -0.04169796,  0.01709518,
        0.08369574,  0.04536458,  0.15168008, -0.11311275, -0.05495327,
        0.04445995,  0.11476301, -0.02229572,  0.0561436 ,  0.08189014,
        0.08566398,  0.11269552,  0.04501221,  0.0729024 , -0.04987087],
      dtype=float32)

In [101]:
results = load_model.docvecs.most_similar([test_vector], topn=2) # 상위 2개까지 가장 비슷한 벡터를 찾아냄

C:\Users\admin\AppData\Local\Temp/ipykernel_8828/851565367.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  results = load_model.docvecs.most_similar([test_vector], topn=2) # 상위 2개까지 가장 비슷한 벡터를 찾아냄


In [102]:
for i in range(0,len(results)):
    index_num = int(results[i][0])
    print(laws_data_list_cut[index_num])
    print("")
    

국민의 권리와 의무  누구든지 위급상황에 처한 경우에는 국가와 지방자치단체로부터 신속한 구조와 구급을 통하여 생활의 안전을 영위할 권리를 가진다   누구든지 119구조대원119구급대원119항공대원이하 구조구급대원이라 한다이 위급상황에서 구조구급활동을 위하여 필요한 협조를 요청하는 경우에는 특별한 사유가 없으면 이에 협조하여야 한다    누구든지 위급상황에 처한 요구조자를 발견한 때에는 이를 지체 없이 소방기관 또는 관계 행정기관에 알려야 하며 119구조대119구급대119항공대이하 구조구급대라 한다가 도착할 때까지 요구조자를 구출하거나 부상 등이 악화되지 아니하도록 노력하여야 한다 

119항공대의 업무 119항공대는 다음 각 호의 업무를 수행한다    인명구조 및 응급환자의 이송의사가 동승한 응급환자의 병원 간 이송을 포함한다   화재 진압   장기이식환자 및 장기의 이송   항공 수색 및 구조 활동   공중 소방 지휘통제 및 소방에 필요한 인력장비 등의 운반   방역 또는 방재 업무의 지원   그 밖에 재난관리를 위하여 필요한 업무

